In [ ]:
'''
Made price into bins
'''
## import packages
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import gc
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import collections
import random

from scipy.sparse import csr_matrix, hstack

from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score

from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import nltk
import itertools
import matplotlib.pyplot as plt
import re

In [ ]:
## import data 
start_time = time.time()

train_raw = pd.read_csv('/home/bsong/Python_Stuff/Data/Kaggle_Mercari/train.tsv',delimiter= '\t')
test_raw = pd.read_csv('/home/bsong/Python_Stuff/Data/Kaggle_Mercari/test.tsv', delimiter='\t')
#train_raw = train_raw.iloc[0:10000,] # just a bit
# standardize price here because may as well
#normalized_price = np.log1p(train_raw['price'].values)
#mean_price_norm = np.mean(normalized_price)
#std_price_norm = np.std(normalized_price) 
#train_raw['price'] = (normalized_price - mean_price_norm)/std_price_norm 

end_time = time.time()
print('import data took ' + str(end_time - start_time) + " seconds.")

In [ ]:
## bins

sort_price = sorted(train_raw.price.values)
perc1 = np.percentile(sort_price,25)
perc2 = np.percentile(sort_price,80)
perc3 = np.percentile(sort_price,90)

total_bins = 4
inds1 = train_raw.train_id[train_raw.price <= perc1].values
inds2 = train_raw.train_id[(train_raw.price > perc1) & (train_raw.price <= perc2)].values
inds3 = train_raw.train_id[(train_raw.price > perc2) & (train_raw.price <= perc3)].values
inds4 = train_raw.train_id[train_raw.price > perc3].values

price_1 = train_raw.price[train_raw.price <= perc1].values
price_2 = train_raw.price[(train_raw.price > perc1) & (train_raw.price <= perc2)].values
price_3 = train_raw.price[(train_raw.price > perc2) & (train_raw.price <= perc3)].values
price_4 = train_raw.price[train_raw.price > perc3].values

train_raw['price_bins'] = [0]*train_raw.shape[0]
train_raw['price_bins_mean'] = [0]*train_raw.shape[0]

train_raw.price_bins.iloc[inds1] = 1
train_raw.price_bins.iloc[inds2] = 2
train_raw.price_bins.iloc[inds3] = 3
train_raw.price_bins.iloc[inds4] = 4

train_raw.price_bins_mean.iloc[inds1] = price_1.mean()
train_raw.price_bins_mean.iloc[inds2] = price_2.mean()
train_raw.price_bins_mean.iloc[inds3] = price_3.mean()
train_raw.price_bins_mean.iloc[inds4] = price_4.mean()

In [ ]:
NUM_BRANDS = 4004
NUM_CATEGORIES = 1001
NAME_MIN_DF = 10
MAX_FEATURES_ITEM_DESCRIPTION = 39000

def handle_missing_inplace(dataset):
    dataset['category_name'].fillna(value='missing', inplace=True)
    dataset['brand_name'].fillna(value='missing', inplace=True)
    dataset['item_description'].fillna(value='missing', inplace=True)


def cutting(dataset):
    pop_brand = dataset['brand_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
    dataset.loc[~dataset['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'
    pop_category = dataset['category_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
    dataset.loc[~dataset['category_name'].isin(pop_category), 'category_name'] = 'missing'


def to_categorical(dataset):
    dataset['category_name'] = dataset['category_name'].astype('category')
    dataset['brand_name'] = dataset['brand_name'].astype('category')
    dataset['item_condition_id'] = dataset['item_condition_id'].astype('category')

## post-prediction functions

def rmsle(h, y): 
    log_h = np.log(h+1) # the +1 is to prevent 0 
    log_y = np.log(y+1) # writing these to prevent memoryerror
    sq_logs = np.square(log_h - log_y)
    score_ = np.sqrt(np.mean(sq_logs))
    return score_

def rmse(h,y):
    sq_logs = np.square(h-y)
    score_ = np.sqrt(np.mean(sq_logs))
    return score_

def unwind(preds, mean_,std_, norm_ = True):
    unstandardized = preds*std_ + mean_
    if norm_ == True: # norm_ is if the original value (like the label) was normalized with np.logm1
        unstandardized = np.expm1(unstandardized)
    return unstandardized

def optimal_weights_ensemble(Xs, y):
    # make sure Xs is all predictions where each column is predictions by each model
    # y just has to be a vector of true values
    # note: the values of Xs need to be scaled to the values that would go into RMSE
    
    y = np.reshape(y,(-1,1)) # to make it shape [n,1]
    
    first = np.matmul(np.transpose(y),Xs)
    second = np.matmul(np.transpose(Xs), Xs)
    w_ = np.matmul(first, np.linalg.inv(second)) # this should be of size [1,n_weights]
    return np.transpose(w_) #returns [n_weight,1] for easier matrix multiplication 

In [ ]:
start_time = time.time()

handle_missing_inplace(train_raw)
print('[{}] Finished to handle missing'.format(time.time() - start_time))

cutting(train_raw)
print('[{}] Finished to cut'.format(time.time() - start_time))

to_categorical(train_raw)
print('[{}] Finished to convert categorical'.format(time.time() - start_time))

cv = CountVectorizer(min_df=NAME_MIN_DF)
X_name = cv.fit_transform(train_raw['name'])
print('[{}] Finished count vectorize `name`'.format(time.time() - start_time))

cv = CountVectorizer()
X_category = cv.fit_transform(train_raw['category_name'])
print('[{}] Finished count vectorize `category_name`'.format(time.time() - start_time))

tv = TfidfVectorizer(max_features=MAX_FEATURES_ITEM_DESCRIPTION,
                     ngram_range=(1, 3),
                     stop_words='english')
X_description = tv.fit_transform(train_raw['item_description'])
print('[{}] Finished TFIDF vectorize `item_description`'.format(time.time() - start_time))

lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(train_raw['brand_name'])
print('[{}] Finished label binarize `brand_name`'.format(time.time() - start_time))

X_dummies = csr_matrix(pd.get_dummies(train_raw[['item_condition_id', 'shipping']],
                                      sparse=True).values)
print('[{}] Finished to get dummies on `item_condition_id` and `shipping`'.format(time.time() - start_time))

sparse_train_raw = hstack((X_dummies, X_description, X_brand, X_category, X_name)).tocsr()
print('[{}] Finished to create sparse train_raw'.format(time.time() - start_time))


In [ ]:
test_ind = round(0.95*sparse_train_raw.shape[0]) # save 5% as a test set 
X_not_test = sparse_train_raw[:test_ind,] # these will be split
Y_not_test = train_raw['price_bins'][:test_ind,]          # into train and validation set

X_test = sparse_train_raw[test_ind:,] # these are the 
Y_test = train_raw['price_bins'][test_ind:,]          # test sets

split_ratio = 0.9
ind_split = round(split_ratio*X_not_test.shape[0])
X_train = X_not_test[:ind_split,]
X_val = X_not_test[ind_split:,]

Y_train = Y_not_test[:ind_split,]
Y_val = Y_not_test[ind_split:,]